In [2]:
import requests
from tqdm import tqdm
import os
import zipfile
import pandas as pd

In [275]:
# US News Historical Data
usnews = "https://andyreiter.com/wp-content/uploads/2022/09/US-News-Rankings-Universities-Through-2023.xlsx"
r = requests.get(usnews, allow_redirects=True)
open('./downloads/usnews.xlsx', 'wb').write(r.content)

37656

#### Initial feature selection

**Features discussed in Reed paper and corresponding tables in 2021:**
- Rank (US News historical data)
- Peer Assessment Score (Historical/Twitter/Page-Rank)
- School (HD2021)
- State (HD2021)
- Public/Private (HD2021)
- Average Freshman Retention Rate (EF2021D)
- Student/Faculty ratio (EF2021D)
- Actual Graduation Rate (GR2021)
- Graduation Rate among PELL recipients (GR2021_PELL_SSL)
- % of Full-time Faculty (EAP2021)
- Faculty salaries (SAL2021_IS)
- SAT/ACT 25th-75th percentile (ADM2021)
- Acceptance Rate (ADM2021)
- Expenditure per FTE student (F2021_F1A, F2021_F2)
- Endowment size per FTE student (F2021_F1A, F2021_F2)

<br>  

**IPEDS Tables to download in 2021:**
- HD2021
- EF2021D
- GR2021
- GR2021_PELL_SSL
- EAP2021
- SAL2021_IS
- ADM2021
- F2021_F1A
- F2021_F2

In [211]:
start_year = 2016
end_year = 2021
IPEDS_files = {}
for current_year in range(start_year, end_year + 1):
    current_files = [
        f"HD{current_year}.zip",
        f"EF{current_year}D.zip",
        f"GR{current_year}.zip",
        f"GR{current_year}_PELL_SSL.zip",
        f"EAP{current_year}.zip",
        f"SAL{current_year}_IS.zip",
        f"ADM{current_year}.zip",
        f"F{current_year-2001}{current_year-2000}_F1A.zip",
        f"F{current_year-2001}{current_year-2000}_F2.zip",
    ]
    IPEDS_files[current_year] = current_files

#### IPEDS File Downloading

In [214]:
url_prefix = "https://nces.ed.gov/ipeds/datacenter/data/"
for current_year in tqdm(range(start_year, end_year + 1), desc="Years"):
    year_dir = f"./downloads/{current_year}/"
    dir_exists = os.path.exists(year_dir)
    if not dir_exists:
        os.makedirs(year_dir)
    for file_name in tqdm(IPEDS_files[current_year], desc=f"{current_year} files"):
        file_path = year_dir + file_name
        if not os.path.exists(file_path):
            r = requests.get(url_prefix + file_name, allow_redirects=True)
            open(year_dir + file_name, 'wb').write(r.content)

Years: 100%|██████████| 6/6 [00:00<00:00, 226.39it/s]


#### IPEDS File Unzipping

In [213]:
for current_year in tqdm(range(start_year, end_year + 1), desc="Years"):
    year_dir = f"./downloads/{current_year}/"
    for zip_file_name in tqdm(IPEDS_files[current_year], desc=f"{current_year} files"):
        csv_file_name = zip_file_name.split(".")[0].lower() + ".csv"
        if not os.path.exists(year_dir + csv_file_name):
            with zipfile.ZipFile(year_dir + zip_file_name, 'r') as zip_ref:
                zip_ref.extractall(year_dir)

Years: 100%|██████████| 6/6 [00:00<00:00, 305.42it/s]


#### IPEDS Feature selection

In [5]:
def latest_csv(file_name_no_ext, csv_set):
    csv_name = file_name_no_ext + ".csv"
    csv_revised_name = file_name_no_ext + "_rv.csv"
    if csv_revised_name in csv_set:
        csv_name = csv_revised_name
    return csv_name

In [216]:
def get_csv_mapping(year_dir, zip_list):
    extracted_csv_files = set(filter(lambda x: x.endswith(".csv"),os.listdir(year_dir)))
    file_name_no_ext_list = [file_name.split(".")[0].lower() for file_name in zip_list]
    return {
        "HD": year_dir + latest_csv(file_name_no_ext_list[0], extracted_csv_files),
        "EF": year_dir + latest_csv(file_name_no_ext_list[1], extracted_csv_files),
        "GR": year_dir + latest_csv(file_name_no_ext_list[2], extracted_csv_files),
        "GR_PELL": year_dir + latest_csv(file_name_no_ext_list[3], extracted_csv_files),
        "EAP": year_dir + latest_csv(file_name_no_ext_list[4], extracted_csv_files),
        "SAL": year_dir + latest_csv(file_name_no_ext_list[5], extracted_csv_files),
        "ADM": year_dir + latest_csv(file_name_no_ext_list[6], extracted_csv_files),
        "F1A": year_dir + latest_csv(file_name_no_ext_list[7], extracted_csv_files),
        "F2": year_dir + latest_csv(file_name_no_ext_list[8], extracted_csv_files),
    }

In [300]:
usnews = pd.read_excel("./downloads/usnews.xlsx", header=1).rename(columns={"IPEDS ID": "UNITID"})
for current_year in tqdm(range(start_year, end_year + 1), desc="Years"):
    year_dir = f"./downloads/{current_year}/"
    csv_files = set(filter(lambda x: x.endswith(".csv"),os.listdir(year_dir)))
    csv_mapping = get_csv_mapping(year_dir, IPEDS_files[current_year])
    
    year_df = usnews[["UNITID", current_year]].rename(columns={current_year: "USNEWSRANK"})
    
    HD = pd.read_csv(csv_mapping["HD"], encoding='latin-1')
    HD = HD[["UNITID", "INSTNM", "IALIAS", "COUNTYNM", "CITY", "STABBR", "ZIP", "WEBADDR", "CONTROL"]]
    year_df = pd.merge(year_df, HD, on='UNITID')

    EF = pd.read_csv(csv_mapping["EF"], encoding='latin-1').rename(columns=lambda x: x.strip())
    EF = EF[["UNITID", "RET_PCF", "STUFACR", "RRFTCT"]].rename(columns={"RET_PCF": "RETENTION", "RRFTCT": "FTCT"})
    year_df = pd.merge(year_df, EF, on='UNITID')

    GR = pd.read_csv(csv_mapping["GR"], encoding='latin-1')
    totals = GR[(GR["CHRTSTAT"] == 12) & (GR["GRTYPE"] == 2)][["UNITID", "GRTOTLT"]].rename(columns={"GRTOTLT": "COHORT"})
    grads = GR[(GR["CHRTSTAT"] == 13) & (GR["GRTYPE"] == 3)][["UNITID", "GRTOTLT"]].rename(columns={"GRTOTLT": "GRADS"})
    GR = pd.merge(totals, grads, on='UNITID')
    GR["GRRATE"] = GR["GRADS"] / GR["COHORT"]
    year_df = pd.merge(year_df, GR, on='UNITID')

    GR_PELL = pd.read_csv(csv_mapping["GR_PELL"], encoding='latin-1')
    GR_PELL = GR_PELL[GR_PELL["PSGRTYPE"] == 2][["UNITID", "PGADJCT", "PGCMBAC"]].rename(columns={"PGADJCT": "PELLCOHORT", "PGCMBAC": "PELLGRADS",})
    GR_PELL["PELLGRRATE"] = GR_PELL["PELLGRADS"] / GR_PELL["PELLCOHORT"]
    year_df = pd.merge(year_df, GR_PELL, on='UNITID')

    EAP = pd.read_csv(csv_mapping["EAP"], encoding='latin-1')
    EAP = EAP[EAP["FACSTAT"] == 10][["UNITID", "EAPFT", "EAPTOT"]].rename(columns={"EAPFT": "FACFT", "EAPTOT": "FACTOT",})
    EAP_SUM = EAP.groupby("UNITID").sum(numeric_only=True)
    EAP_SUM["FTPCT"] = EAP_SUM["FACFT"] / EAP_SUM["FACTOT"]
    year_df = pd.merge(year_df, EAP_SUM, on='UNITID')
    
    SAL = pd.read_csv(csv_mapping["SAL"], encoding='latin-1').groupby("UNITID").sum(numeric_only=True)
    SAL = SAL[["SA09MAT"]].rename(columns={"SA09MAT":"AVGSAL"})
    year_df = pd.merge(year_df, SAL, on='UNITID')
    
    ADM = pd.read_csv(csv_mapping["ADM"], encoding='latin-1')
    ADM = ADM[["UNITID", "SATVR25", "SATVR75", "SATMT25", "SATMT75", "ACTCM25", "ACTCM75", "ADMSSN", "APPLCN"]]
    ADM["ACPTRT"] = ADM["ADMSSN"] / ADM["APPLCN"]
    year_df = pd.merge(year_df, ADM, on='UNITID')
    
    F1A = pd.read_csv(csv_mapping["F1A"], encoding='latin-1').rename(columns=lambda x: x.strip())
    F1A = F1A[["UNITID", "F1C011", "F1C021", "F1C031", "F1C051", "F1C061", "F1C071", "F1H02"]].rename(columns={
        "F1C011": "EINSTRUCTIONAL",
        "F1C021": "ERESEARCH",
        "F1C031": "EPUBLIC",
        "F1C051": "EACADEMIC",
        "F1C061": "ESTUDENT",
        "F1C071": "EINSTITUTIONAL",
        "F1H02": "ENDOWMENT"
    })
    F2 = pd.read_csv(csv_mapping["F2"], encoding='latin-1').rename(columns=lambda x: x.strip())
    F2 = F2[["UNITID", "F2E011", "F2E021", "F2E031", "F2E041", "F2E051", "F2E061", "F2H02"]].rename(columns={
        "F2E011": "EINSTRUCTIONAL",
        "F2E021": "ERESEARCH",
        "F2E031": "EPUBLIC",
        "F2E041": "EACADEMIC",
        "F2E051": "ESTUDENT",
        "F2E061": "EINSTITUTIONAL",
        "F2H02": "ENDOWMENT"
    })
    FINANCE = pd.concat([F1A, F2], ignore_index=True, axis=0)
    year_df = pd.merge(year_df, FINANCE, on='UNITID')
    year_df["EINSTRUCTIONAL"] = year_df["EINSTRUCTIONAL"] / year_df["FTCT"]
    year_df["ERESEARCH"] = year_df["ERESEARCH"] / year_df["FTCT"]
    year_df["EPUBLIC"] = year_df["EPUBLIC"] / year_df["FTCT"]
    year_df["EACADEMIC"] = year_df["EACADEMIC"] / year_df["FTCT"]
    year_df["ESTUDENT"] = year_df["ESTUDENT"] / year_df["FTCT"]
    year_df["EINSTITUTIONAL"] = year_df["EINSTITUTIONAL"] / year_df["FTCT"]
    year_df["ENDOWMENT"] = year_df["ENDOWMENT"] / year_df["FTCT"]

    output_path = f"./by_year/{current_year}.csv"
    year_df.to_csv(output_path, index=False)

Years: 100%|██████████| 6/6 [00:05<00:00,  1.02it/s]


In [301]:
# Reed College
year_df[year_df["UNITID"] == 209922]

,UNITID,USNEWSRANK,INSTNM,IALIAS,COUNTYNM,CITY,STABBR,ZIP,WEBADDR,CONTROL,...,ADMSSN,APPLCN,ACPTRT,EINSTRUCTIONAL,ERESEARCH,EPUBLIC,EACADEMIC,ESTUDENT,EINSTITUTIONAL,ENDOWMENT


In [302]:
# Rice
year_df[year_df["UNITID"] == 227757]

,UNITID,USNEWSRANK,INSTNM,IALIAS,COUNTYNM,CITY,STABBR,ZIP,WEBADDR,CONTROL,...,ADMSSN,APPLCN,ACPTRT,EINSTRUCTIONAL,ERESEARCH,EPUBLIC,EACADEMIC,ESTUDENT,EINSTITUTIONAL,ENDOWMENT
15,227757,16.0,Rice University,,Harris County,Houston,TX,77005-1827,www.rice.edu/,2,...,2802.0,29544,0.094842,385114.559919,166601.878147,0.0,45049.007049,86138.364552,37134.339376,8.137253e+06
